# Modelli: training con RandomForest e LightGBM in ensemble

# Imports 

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

# Parte Princiaple
La funzione: applica una Stratified K-Fold CV per evitare bias in problemi sbilanciati, allena due modelli (RandomForestClassifier e LGBMClassifier) su ciascun fold, crea un ensemble "soft" scegliendo, per ciascun esempio, la predizione del modello con la confidenza maggiore (valore massimo tra le probabilità), ritorna tutte le predizioni combinate da tutti i fold, utili per calcolare metriche globali.

In [ ]:
def train_ensemble(X, y):
    # Inizializza un classificatore Random Forest con iperparametri specifici
    rf = RandomForestClassifier(
        n_estimators=953,
        max_features=0.5,
        min_samples_split=9,
        min_samples_leaf=3,
        bootstrap=True,
        n_jobs=-1,
        random_state=2025
    )
    # Inizializza un classificatore LightGBM con iperparametri ottimizzati
    lgbm = LGBMClassifier(
        n_estimators=800,
        learning_rate=0.06,
        num_leaves=160,
        max_depth=None,
        min_child_samples=60,
        subsample=0.75,
        colsample_bytree=0.5,
        reg_alpha=0.5,
        reg_lambda=0.026,
        verbosity=-1,
        random_state=42,
        n_jobs=-1
    )
    # Imposta una cross-validation stratificata a 5 fold
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)
    # Lista per raccogliere tutte le predizioni del processo di validazione
    preds = []
     # Loop su ciascun fold della cross-validation
    for train_idx, val_idx in kf.split(X, y):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        # Addestra entrambi i modelli sul fold corrente
        rf.fit(X_tr, y_tr)
        lgbm.fit(X_tr, y_tr)
        # Calcola le probabilità di classe per il set di validazione
        proba_rf = rf.predict_proba(X_val)
        proba_lgbm = lgbm.predict_proba(X_val)
        fold_preds = []
        # Ensemble delle predizioni: seleziona il modello con la probabilità massima più alta
        for i in range(len(X_val)):
            p_rf = proba_rf[i]
            p_lg = proba_lgbm[i]
            # Estrae la classe predetta da ciascun modello
            pred_rf = rf.classes_[np.argmax(p_rf)]
            pred_lg = lgbm.classes_[np.argmax(p_lg)]
            # Aggiunge la predizione del modello più sicuro
            fold_preds.append(pred_lg if np.max(p_lg) > np.max(p_rf) else pred_rf)
        preds.extend(fold_preds)
    # Ritorna tutte le predizioni ottenute in validazione durante la cross-validation
    return preds
